In [9]:
import pickle
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime
import glob
from sklearn import preprocessing

from sklearn.preprocessing import OneHotEncoder as ohc

"""sliding window에서 stepSize가 sliding 1초에 할건지 3초에할건지인지 지금은 3초로 되있음 """
def sliding_window(data,stepSize,windowSize):
    for x in range(0,len(data)-windowSize+1,stepSize):
        yield(x,data[x:x+windowSize])

"""3초씩 슬아디잉하고 여기서 데이터는 10분짜리 complex데이터임 6초 window를 3초씩 슬라이딩한 결과 (199,180)짜리 test set이 나옴
여기서 180은 simpled activity에서 feature가 5개이고, 36개 acc, gyro list에서 피처뽑아서 5*180
그걸 바로 clf로 인퍼런스하는게 출력"""
def pred_window(clf,data):
    test_set=[]
    for(x,window) in sliding_window(data,3,6): 
        wind=window.reshape(180)
        test_set.append(wind)
    #print(np.shape(test_set))
    return clf.predict(test_set)

"""pred_window랑 똑같은데 출력만 다름 확률뽑을때 predict하기 전 test_set이 필요해서 """
def test_set(data):
    test_set=[]
    for(x,window) in sliding_window(data,3,6): 
        wind=window.reshape(180)
        test_set.append(wind)
    return (test_set)

#filename queue
filename_queue=glob.glob('./featuredata/compfeature[1-2].csv')
data=list()
context_label=list()
context_label_count=np.zeros(4)

for filename in filename_queue:
    print(filename)
    file=open(filename, newline='')
    reader=csv.reader(file)
    #context=reader[seq(1, nrow(reader), 2),]
    #data = reader[seq(1, nrow(reader), 2),]
    i=0
    array=list()


    for row in reader:
        i+=1
        if(int(i)%2==1):
            context_label.append(row)
            context_label_count[int(row[0])]+=1
        else:
            array=np.array(row)
            array=array.reshape(600,6,5)
            data.append(array)


./featuredata/compfeature2.csv
./featuredata/compfeature1.csv


In [10]:
# load svm
filename='./svm.sav'
svm = pickle.load(open(filename, 'rb'))

# load knn
filename='./knn.sav'
knn = pickle.load(open(filename, 'rb'))

# load rf
filename='./rf.sav'
rf = pickle.load(open(filename, 'rb'))

# load ETC
filename='./ETC.sav'
ETC = pickle.load(open(filename, 'rb'))

In [11]:
"""
이거는 그냥 볼려고 해본거구 무시하셈
pred=[]
for x in data: #10 minute granularity
    pred.append(pred_window(ETC,x))
print(context_label_count)
print(context_label)
print(pred)
"""

'\n이거는 그냥 볼려고 해본거구 무시하셈\npred=[]\nfor x in data: #10 minute granularity\n    pred.append(pred_window(ETC,x))\nprint(context_label_count)\nprint(context_label)\nprint(pred)\n'

라벨-액티비티
0 얼굴만지기
1 필기하기
2 건배하기
3 마시기1
4 휴지뽑기
5 마시기2
6 숟가락
7 젓가락 
8 포크
9 젓기
16 idle

빠진 액티비티 키보드, 마우스, 손들기, 사진찍기, 술 따르기, 스마트폰 타이핑

In [12]:
"""이게 ETC complex prediction 확률로 0.3미만은 idle(=16)처리할거임"""
from sklearn import preprocessing

pred=list()
pred2=[] ##이게 prediction
test_pred_proba=[]
for x in data: #10 minute granularity
    test_pred_proba.append(ETC.predict_proba(preprocessing.quantile_transform(test_set(x)))) 
    """헐 여기 프리프로세싱 들어감..."""
print(np.shape(test_pred_proba))
for i in test_pred_proba:
    for j in i:
        if np.max(j)<0.3: #확률 0.3미만이면 idle
            pred.append(10)
        else:
            pred.append(np.argmax(j))
"""여긴 그냥 사이즈 맞춰줄려고 199길이 리스트에다가 +'n'추가할거임"""
for i in range(int(len(pred)/199)):
    pred2.append(pred[199*i:199*i+199])
for i in pred2:
    i.append(10)
pred2=np.array(pred2)
pred2=pred2.reshape(len(data),10,20)

print(context_label)
print(pred2)
"""21,199,10은 test_pred_proba의 shape인데 이게 뭐나면 라벨이 10개여서 마지막에 10이고 각각의 확률 나오는거임
21은 현재까지 모인 Context 데이터 개수(쏘리 말잘못했음 전화할때) 199는 10분을 window Sliding 하면 199나옴"""

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2553: FutureWarning: The default value of `copy` will change from False to True in 0.23 in order to make it more consistent with the default `copy` values of other functions in :mod:`sklearn.preprocessing.data` and prevent unexpected side effects by modifying the value of `X` inplace. To avoid inplace modifications of `X`, it is recommended to explicitly set `copy=True`
  FutureWarning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2239: UserWarning: n_quantiles (1000) is greater than the total number of samples (199). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2553: FutureWarning: The default value of `copy` will change from False to True in 0.23 in order to make it more consistent with the default `copy` values of other functions in :mo

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2553: FutureWarning: The default value of `copy` will change from False to True in 0.23 in order to make it more consistent with the default `copy` values of other functions in :mod:`sklearn.preprocessing.data` and prevent unexpected side effects by modifying the value of `X` inplace. To avoid inplace modifications of `X`, it is recommended to explicitly set `copy=True`
  FutureWarning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2239: UserWarning: n_quantiles (1000) is greater than the total number of samples (199). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2553: FutureWarning: The default value of `copy` will change from False to True in 0.23 in order to make it more consistent with the default `copy` values of other functions in :mo

(21, 199, 10)
[['1'], ['1'], ['2'], ['3'], ['1'], ['2'], ['1'], ['1'], ['1'], ['1'], ['2'], ['2'], ['3'], ['3'], ['1'], ['1'], ['2'], ['2'], ['1'], ['3'], ['3']]
[[[10 10 10 ... 10 10  0]
  [10 10 10 ...  8  8  8]
  [10  8  8 ...  7  7  7]
  ...
  [ 8 10 10 ...  3  3  3]
  [10  3 10 ...  8  8  2]
  [10  8 10 ...  8  8 10]]

 [[ 8 10  7 ... 10  3  3]
  [ 3  3 10 ... 10  8 10]
  [ 8  8 10 ... 10  8  8]
  ...
  [ 2 10 10 ... 10 10 10]
  [10  8 10 ...  7 10 10]
  [10  8  8 ... 10 10 10]]

 [[10  0  0 ...  8  8  8]
  [10  8  8 ... 10 10 10]
  [ 0  0  0 ...  8  8  8]
  ...
  [10  0  0 ...  8  8  8]
  [ 8  8  8 ... 10 10  8]
  [ 8  8  8 ... 10  8 10]]

 ...

 [[10 10 10 ... 10 10  8]
  [10  8 10 ...  8  2 10]
  [10  8 10 ...  8  8  3]
  ...
  [10  8 10 ... 10 10 10]
  [10 10  7 ... 10  8  8]
  [ 8 10 10 ... 10 10 10]]

 [[10 10  2 ...  8  8 10]
  [ 8 10 10 ...  8 10 10]
  [10  2  8 ...  8  8  8]
  ...
  [10  0  0 ... 10 10 10]
  [10 10  8 ...  8 10  8]
  [10  7  7 ... 10 10 10]]

 [[10 10  2 

'21,199,10은 test_pred_proba의 shape인데 이게 뭐나면 라벨이 10개여서 마지막에 10이고 각각의 확률 나오는거임\n21은 현재까지 모인 Context 데이터 개수(쏘리 말잘못했음 전화할때) 199는 10분을 window Sliding 하면 199나옴'

In [13]:
### Context prediction 용 데이터 만들기 길이 (라벨10개+idle1개)11*(10분)10 = 110
context_data=[]
for context in pred2:
    context_=[]
    for minute in context:
        activity_count=np.zeros(11)
        for activity in minute:
            activity_count[activity]+=1
        context_.append(activity_count)
        context=list(context)
    context_data.append(context_)
context_data=np.array(context_data)
context_data=context_data.reshape(len(context_data),110)


In [14]:
print(np.shape(context_data))
print(np.shape(context_label))
print(context_label)

(21, 110)
(21, 1)
[['1'], ['1'], ['2'], ['3'], ['1'], ['2'], ['1'], ['1'], ['1'], ['1'], ['2'], ['2'], ['3'], ['3'], ['1'], ['1'], ['2'], ['2'], ['1'], ['3'], ['3']]


In [15]:
X=context_data
y=context_label


In [16]:
### 여기서부터 context prediction 시작
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

clf=RandomForestClassifier()
clf.fit(X,y)

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [17]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score 
scores=cross_val_score(clf,X,y,cv=3,scoring='accuracy')
print(scores.mean())

1.0


/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


In [ ]:
from joblib import dump, load

filename='./complex_rf.sav'
pickle.dump(clf,open(filename,'wb'))
dump(clf,'complex_rf.joblib')

In [ ]:
clf=KNeighborsClassifier()
clf.fit(X,y)

In [ ]:

scores=cross_val_score(clf,X,y,cv=3,scoring='accuracy')
print(scores.mean())